# Possion equation in 2D

## `PoissonSolver2D` class

The class `PoissonSolver2D` provides a solver for the Poisson equation in two dimensions:
$$
    \begin{aligned}
		- \Delta u (x,y) & = 2 a \pi^2 \sin (\pi x) \sin (\pi y) & & \text{in } \Omega, \\
		u(x,y) & = b xy & & \text{on } \partial\Omega, 
	\end{aligned}
$$
where $\Omega = (0,1)^2$ is the unit square. The solver uses a finite difference method on a uniform grid. Here, $a$ and $b$ are variable parameters.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve

class PoissonSolver2D:
    def __init__(self, nx, ny, lx, ly):
        """
        Initialize the Poisson solver with grid size and domain dimensions.
        
        Parameters:
        nx (int): Number of grid points in the x-direction.
        ny (int): Number of grid points in the y-direction.
        lx (float): Length of the domain in the x-direction.
        ly (float): Length of the domain in the y-direction.
        """
        self.nx = nx
        self.ny = ny
        self.lx = lx
        self.ly = ly
        self.dx = lx / (nx - 1)
        self.dy = ly / (ny - 1)
        self.u = np.zeros((ny, nx))  # Solution array
        self.f = np.zeros((ny, nx))  # Source term array

    def set_boundary_conditions(self, boundary_func):
        """
        Set the boundary conditions using a provided function.
        
        Parameters:
        boundary_func (function): Function that takes (x, y) and returns the boundary value.
        """
        for i in range(self.nx):
            self.u[0, i] = boundary_func(i * self.dx, 0)  # Bottom boundary
            self.u[-1, i] = boundary_func(i * self.dx, self.ly)  # Top boundary
        for j in range(self.ny):
            self.u[j, 0] = boundary_func(0, j * self.dy)  # Left boundary
            self.u[j, -1] = boundary_func(self.lx, j * self.dy)  # Right boundary

    def set_source_term(self, source_func):
        """
        Set the source term using a provided function.
        
        Parameters:
        source_func (function): Function that takes (x, y) and returns the source term value.
        """
        for j in range(1, self.ny - 1):
            for i in range(1, self.nx - 1):
                self.f[j, i] = source_func(i * self.dx, j * self.dy)

    def setup_linear_system(self):
        """
        Set up the linear system for the Poisson equation.
        
        Returns:
        A (scipy.sparse.lil_matrix): Sparse matrix representing the linear system.
        b (numpy.ndarray): Right-hand side vector.
        """
        n = self.nx * self.ny
        A = lil_matrix((n, n))
        b = np.zeros(n)

        for j in range(self.ny):
            for i in range(self.nx):
                idx = j * self.nx + i
                if i == 0 or i == self.nx - 1 or j == 0 or j == self.ny - 1:
                    A[idx, idx] = 1
                    b[idx] = self.u[j, i]
                else:
                    A[idx, idx] = -4
                    A[idx, idx - 1] = 1
                    A[idx, idx + 1] = 1
                    A[idx, idx - self.nx] = 1
                    A[idx, idx + self.nx] = 1
                    b[idx] = - (self.dx*self.dy) * self.f[j, i]

        return A, b

    def solve(self):
        """
        Solve the Poisson equation using a sparse direct solver.
        """
        A, b = self.setup_linear_system()
        u_flat = spsolve(A.tocsr(), b)
        self.u = u_flat.reshape((self.ny, self.nx))

    def get_solution(self):
        """
        Get the solution of the Poisson equation.
        
        Returns:
        numpy.ndarray: Solution array.
        """
        return self.u
    
    def get_meshgrid(self):
        """
        Get the meshgrid for the domain.
        
        Returns:
        numpy.ndarray: Meshgrid for the domain.
        """
        x = np.linspace(0, self.lx, self.nx)
        y = np.linspace(0, self.ly, self.ny)
        return np.meshgrid(x, y)
    
    def plot_solution(self):
        """
        Plot the solution of the Poisson equation.
        """
        plt.imshow(self.u, origin='lower', extent=(0, self.lx, 0, self.ly))
        plt.colorbar()
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Poisson equation solution')
        plt.show()

## Example script

In the following script, the boundary value problem 
$$
    \begin{aligned}
		- \Delta u (x,y) & = 2 \pi^2 \sin (\pi x) \sin (\pi y) & & \text{in } \Omega, \\
		u(x,y) & = 0 & & \text{on } \partial\Omega, 
	\end{aligned}
$$
is solved on a grid with $100 \times 100$ elements and the solution is plotted.

In [ ]:
# Define the boundary condition function
def boundary_func(x, y):
    return 0.0

# Define the source term function
def source_func(x, y):
    return 2.0 * np.pi**2 * np.sin(np.pi * x) * np.sin(np.pi * y)

# Create an instance of the PoissonSolver2D class
solver = PoissonSolver2D(nx=100, ny=100, lx=1.0, ly=1.0)

# Set the boundary conditions
solver.set_boundary_conditions(boundary_func)

# Set the source term
solver.set_source_term(source_func)

# Setup the linear system
solver.setup_linear_system()

# Solve the Poisson equation
solver.solve()

# Plot the solution
solver.plot_solution()